In [1]:
# dependencies
import pandas as pd
import os

In [2]:
# set file paths
schoolsPath = os.path.join("Resources", "schools_complete.csv")
studentsPath = os.path.join("Resources", "students_complete.csv")

# read in csv files
schools = pd.read_csv(schoolsPath)
students = pd.read_csv(studentsPath)

In [38]:
# combine both files into one csv dataset
schoolNstudents = pd.merge(students, schools, how="left", on=["school_name", "school_name"])

In [4]:
# calculate the total number of schools
unique_schools = schoolNstudents["school_name"].unique()
total_schools = len(unique_schools)
total_schools

15

In [5]:
# calculate the total number of students
total_students = len(schoolNstudents["student_name"])
total_students

39170

In [6]:
# calculate the total budget
total_budget = schools["budget"].sum()
total_budget

24649428

In [7]:
# calculate the average math score
average_math = students["math_score"].mean()
average_math

78.98537145774827

In [8]:
# calculate the average reading score
average_reading = students["reading_score"].mean()
average_reading

81.87784018381414

In [9]:
# calculate the percentage of students with a passing math score (70 or greater)
math_greater_70 = students.loc[students["math_score"] >= 70, :]
math_passing_percent = (len(math_greater_70) / total_students)*100
math_passing_percent

74.9808526933878

In [10]:
# calculate the percentage of students with a passing reading score (70 or greater)
reading_greater_70 = students.loc[students["reading_score"] >= 70, :]
reading_passing_percent = (len(reading_greater_70) / total_students)*100
reading_passing_percent

85.80546336482001

In [11]:
# calculate the percentage of students who passed math and reading (% Overall Passing)
passing_math_reading = students.loc[(students["math_score"] >= 70) & (students["reading_score"] >= 70), :]
overall_passing_percent = (len(passing_math_reading) / total_students)*100
overall_passing_percent

65.17232575950983

In [12]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                        "Total Students": [total_students],
                        "Total Budget": [total_budget],
                        "Average Math Score": [average_math],
                        "Average Reading Score": [average_reading],
                        "% Passing Math": [math_passing_percent],
                        "% Passing Reading": [reading_passing_percent],
                        "% Overall Passing": [overall_passing_percent],
                        })

In [13]:
# converting integers to float for formatting
district_summary["Total Students"] = district_summary["Total Students"].astype(float)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float)

In [14]:
# formatting for better read of district summary
district_summary["Total Students"] = district_summary["Total Students"].map('{:,.0f}'.format)
district_summary["Total Budget"] = district_summary["Total Budget"].map('${:,.2f}'.format)
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


In [55]:
schools["per_student_budget"] = ((schools["budget"] / schools["size"]).map('${:,.2f}'.format))